<a href="https://colab.research.google.com/github/ivianrr/curso-big-data/blob/main/Sesi%C3%B3n%203/Prophet_Predecir_el_valor_de_Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://shorturl.at/mqvCJ

# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [30]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [31]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [32]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2022-11-02'
start_date = '2018-01-01'
# Descargar el dataframe
commodity='HG=F'
nflx_df = yf.download(commodity,start_date, today).reset_index()

nflx_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1214,2022-10-26,$3.56,$3.63,$3.56,$3.63,$3.63,367
1215,2022-10-27,$3.60,$3.63,$3.60,$3.63,$3.63,761
1216,2022-10-28,$3.54,$3.54,$3.45,$3.47,$3.47,860
1217,2022-10-31,$3.41,$3.43,$3.41,$3.41,$3.41,587
1218,2022-11-01,$3.49,$3.52,$3.48,$3.51,$3.51,405


In [33]:
# Validamos que no hay datos vacíos
nflx_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [34]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = nflx_df[["Date", "Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns=new_names)

df.head()

,ds,y
0,2018-01-02,$3.25
1,2018-01-03,$3.24
2,2018-01-04,$3.26
3,2018-01-05,$3.21
4,2018-01-08,$3.21


In [35]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [36]:
df.tail()

,ds,y
1214,2022-10-26,$3.56
1215,2022-10-27,$3.60
1216,2022-10-28,$3.54
1217,2022-10-31,$3.41
1218,2022-11-01,$3.49


In [37]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = f"Serie temporal de {commodity}"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [38]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode="multiplicative")

In [39]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [40]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365, include_history=True)
future = future[ future['ds'].dt.dayofweek < 5]
future.head(10)

,ds
0,2018-01-02
1,2018-01-03
2,2018-01-04
3,2018-01-05
4,2018-01-08
5,2018-01-09
6,2018-01-10
7,2018-01-11
8,2018-01-12
9,2018-01-16


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [41]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [42]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1475,2023-10-26,$2.57,$-1.10,$6.25
1476,2023-10-27,$2.56,$-1.17,$6.23
1477,2023-10-30,$2.55,$-1.14,$6.26
1478,2023-10-31,$2.54,$-1.19,$6.29
1479,2023-11-01,$2.53,$-1.17,$6.25


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [44]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
# forecast=forecast[forecast['yhat']>0]
# forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)
# plotly.gca() 
# title=f"Predicción de los valores en un año {commodity}"

In [45]:
plot_components_plotly(m, forecast)
# Le ponemos el título a la gráfica
titulo = f"Trens de de {commodity}"
fig.update_layout(
    title_text=titulo,
)